# Python Logging in Klassen

Da wir Objektorientiert Arbeiten, sollten wir nicht in jeder Klasse einen Logger initialisieren, sondern den Logger in eine eigene Klasse auslagern, denn wir wollen doppelten Code vermeiden!

In [8]:
import logging
from logging.handlers import RotatingFileHandler

class Logger:

    def __init__(self, name, log_file, max_size=10*1024*1024, backup_count=5):
        self.logger = logging.getLogger(name)
        self.logger.setLevel(logging.DEBUG)
        handler = RotatingFileHandler(log_file, maxBytes=max_size, backupCount=backup_count)
        handler.setLevel(logging.DEBUG)
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)

    def debug(self, msg):
        self.logger.debug(msg)

    def info(self, msg):
        self.logger.info(msg)

    def warning(self, msg):
        self.logger.warning(msg)

    def error(self, msg, exc_info=False):
        self.logger.error(msg, exc_info=exc_info)

    def critical(self, msg):
        self.logger.critical(msg)

    def remove_handler(self):
        print(f"Removing handler {self.logger.name}")
        self.logger.removeHandler(self.logger.handlers[0])	

    def __repr__(self):
        return f"Logger({self.logger.name})"

In [20]:
from helper import DbMock
import time

class Calculator:
    def __init__(self, divident:int, divisor:int):
        self.logger = Logger(__class__.__name__, "calculator.log")
        self.divident = divident
        self.divisor = divisor

    def run(self):
        self._divide()
           
    def _divide(self):
        try:
            self.logger.info(f"divide {self.divident} / {self.divisor}")
            quotient = self.divident / self.divisor
            self.logger.info(f"{self.divident} / {self.divisor} = {quotient}")
            return quotient
        except ZeroDivisionError:
            self.logger.error(f"beim teilen von ({self.divident} / {self.divisor}) ist ein fehler aufgetreten:", exc_info=True)
           
    def __del__(self):
        self.logger.remove_handler()
        print(f"{__class__.__name__} wurde Zerstört")


class SQLConnection:
    def __init__(self, url:str):
        self.logger = Logger(__class__.__name__, "sql_connection.log")
        self.url = url

    def run(self):
        self._connect()

    def _connect(self):
        try:
            DbMock.connect(self.url)
        except Exception as ex:
            self.logger.error(ex, exc_info=True)
       
    def __del__(self):
        self.logger.remove_handler()
        print(f"{__class__.__name__} wurde Zerstört")


print("Starte Calc")
calc = Calculator(20,0)
print(f"calc.logger.__repr__(): {calc.logger.__repr__()}")
calc.run()
print("Ende Calc")
print("Starte SQL")
sql = SQLConnection(1)
print(f"sql.logger.__repr__(): {sql.logger.__repr__()}")
sql.run()
print("Ende SQL")


Starte Calc
Removing handler Calculator
Calculator wurde Zerstört
calc.logger.__repr__(): Logger(Calculator)
Ende Calc
Starte SQL
Removing handler SQLConnection
SQLConnection wurde Zerstört
sql.logger.__repr__(): Logger(SQLConnection)
Ende SQL


In [21]:
from helper import DbMock
import threading
import time

class Calculator(threading.Thread):
    def __init__(self, divident:int, divisor:int):
        super(Calculator, self).__init__()
        self.logger = Logger(__class__.__name__, "calculator.log")
        self._stop = threading.Event()
        self.divident = divident
        self.divisor = divisor

    def stop(self):
        self._stop.set()

    def _stopped(self):
        return self._stop.is_set()

    def run(self):
        while True:
            self._divide()
            if self._stopped():
                return

    def _divide(self):
        try:
            self.logger.info(f"divide {self.divident} / {self.divisor}")
            quotient = self.divident / self.divisor
            self.logger.info(f"{self.divident} / {self.divisor} = {quotient}")
            return quotient
        except ZeroDivisionError:
            self.logger.error(f"beim teilen von ({self.divident} / {self.divisor}) ist ein fehler aufgetreten:", exc_info=True)
           
    def __del__(self):
        self.logger.remove_handler()
        print(f"{__class__.__name__} wurde Zerstört")


class SQLConnection(threading.Thread):
    def __init__(self, url:str):
        super(SQLConnection, self).__init__()
        self.logger = Logger(__class__.__name__, "sql_connection.log")
        self._stop = threading.Event()
        self.url = url

    def stop(self):
        self._stop.set()

    def _stopped(self):
        return self._stop.is_set()

    def run(self):
        while True:
            self._connect()
            if self._stopped():
                return

    def _connect(self):
        try:
            DbMock.connect(self.url)
        except Exception as ex:
            self.logger.error(ex, exc_info=True)
       
    def __del__(self):
        self.logger.remove_handler()
        print(f"{__class__.__name__} wurde Zerstört")

thread_1 = Calculator(20,0)
thread_2 = SQLConnection(1)

thread_1.start()
thread_2.start()
time.sleep(60)
print("stopping threads")
thread_1.stop()
thread_2.stop()
time.sleep(2)
thread_1.__del__()
thread_2.__del__()


stopping threads
Removing handler Calculator
Calculator wurde Zerstört
Removing handler SQLConnection
SQLConnection wurde Zerstört
